In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, FloatSlider, Button, VBox, HBox, Output, FloatText, Dropdown

PRECIO_LITRO_GASOLINA = 24.0
COSTO_KWH = 2.8
COSTO_M3_AGUA = 20.0
KGCO2_POR_LITRO = 2.31

try:
    df = pd.read_csv("/content/BaseDatos_SmartCity_Greedy_V3.csv")
    print("Base de datos cargada correctamente.")
except FileNotFoundError:
    print("No se encontró la base. Generando datos sintéticos.")
    np.random.seed(42)
    df = pd.DataFrame({
        "Nombre": [f"Usuario{i}" for i in range(1, 41)],
        "GastoGasolina": np.random.randint(500, 8000, 40),
        "GastoLuz": np.random.randint(200, 3000, 40),
        "GastoAgua": np.random.randint(100, 1500, 40),
        "RendimientoKmL": np.random.uniform(8, 20, 40),
        "CO2porKm": np.random.uniform(0.08, 0.28, 40)
    })

def co2_from_gas_mxn(mxn, co2_km):
    litros = mxn / PRECIO_LITRO_GASOLINA
    return litros * KGCO2_POR_LITRO * (co2_km / 0.20)

def kwh_from_mxn(mxn): return mxn / COSTO_KWH
def m3_from_mxn(mxn):  return mxn / COSTO_M3_AGUA

out = Output()
meta_state = {"total": 400, "gas": 300, "agua": 70, "luz": 30}
gasto_real = {"gasolina": 0.0, "agua": 0.0, "luz": 0.0}

#Actualización de deslizadores y visualización
def sim_usuario(nombre_usuario, meta_total, ahorro_gas, ahorro_agua):
    fila = df[df["Nombre"] == nombre_usuario].iloc[0]
    gasto_base_gasolina = fila["GastoGasolina"]
    gasto_base_luz = fila["GastoLuz"]
    gasto_base_agua = fila["GastoAgua"]
    co2_km = fila["CO2porKm"]
    rend_kml = fila["RendimientoKmL"]

    #Ajuste de límites y cálculos
    ahorro_gas = min(ahorro_gas, meta_total)
    restante = meta_total - ahorro_gas
    ahorro_agua = min(ahorro_agua, restante)
    ahorro_luz = meta_total - ahorro_gas - ahorro_agua
    meta_state.update({"total": meta_total, "gas": ahorro_gas, "agua": ahorro_agua, "luz": ahorro_luz})

    #Porcentajes
    pct_gas = ahorro_gas / gasto_base_gasolina * 100
    pct_agua = ahorro_agua / gasto_base_agua * 100
    pct_luz = ahorro_luz / gasto_base_luz * 100

    #Impacto físico
    co2_ev = co2_from_gas_mxn(ahorro_gas, co2_km)
    kwh_ev = kwh_from_mxn(ahorro_luz)
    agua_ev_l = m3_from_mxn(ahorro_agua) * 1000

    with out:
        out.clear_output()
        fig, ax = plt.subplots(1, 2, figsize=(10,4))
        rubros = ["Gasolina", "Luz", "Agua"]
        valores = [ahorro_gas, ahorro_luz, ahorro_agua]
        ax[0].bar(rubros, valores, color=["#FF9800","#2196F3","#00BCD4"])
        ax[0].set_title("Distribución de ahorro planeado (MXN)")
        ax[0].set_ylim(0, meta_total*1.1)

        metricas = ["CO₂ evitado (kg)", "kWh evitados", "Agua evitada (L)"]
        impacto = [co2_ev, kwh_ev, agua_ev_l]
        ax[1].bar(metricas, impacto, color=["#f44336","#3F51B5","#4CAF50"])
        ax[1].set_title("Impacto ecológico estimado")
        plt.tight_layout()
        plt.show()

        print(f"Usuario: {nombre_usuario}")
        print(f"Meta total: {meta_total:.0f} MXN")
        print(f"Gasolina: {ahorro_gas:.0f} ({pct_gas:.1f}%)")
        print(f"Agua: {ahorro_agua:.0f} ({pct_agua:.1f}%)")
        print(f"Luz: {ahorro_luz:.0f} ({pct_luz:.1f}%)")
        print(f"\n Impacto estimado:")
        print(f"CO₂ evitado: {co2_ev:.2f} kg")
        print(f"Energía evitada: {kwh_ev:.1f} kWh")
        print(f"Agua evitada: {agua_ev_l:.0f} L")

# Registrar gastos reales y recalcular ahorros
def registrar_gasto(cat, monto, nombre_usuario):
    fila = df[df["Nombre"] == nombre_usuario].iloc[0]
    co2_km = fila["CO2porKm"]

    gasto_real[cat] += monto
    ahorro_logrado = {
        "gas": max(0, meta_state["gas"] - gasto_real["gasolina"]),
        "agua": max(0, meta_state["agua"] - gasto_real["agua"]),
        "luz": max(0, meta_state["luz"] - gasto_real["luz"])
    }

    co2_real = co2_from_gas_mxn(gasto_real["gasolina"], co2_km)
    kwh_real = kwh_from_mxn(gasto_real["luz"])
    agua_real_l = m3_from_mxn(gasto_real["agua"]) * 1000

    total_restante = ahorro_logrado["gas"] + ahorro_logrado["agua"] + ahorro_logrado["luz"]

    with out:
        out.clear_output(wait=True)
        fig, ax = plt.subplots(1, 2, figsize=(10,4))
        rubros = ["Gasolina", "Luz", "Agua"]
        valores = [ahorro_logrado["gas"], ahorro_logrado["luz"], ahorro_logrado["agua"]]
        ax[0].bar(rubros, valores, color=["#FF9800","#2196F3","#00BCD4"])
        ax[0].set_title("Ahorro restante (MXN)")
        ax[0].set_ylim(0, meta_state["total"])

        metricas = ["CO₂ emitido (kg)", "kWh usados", "Agua usada (L)"]
        impacto = [co2_real, kwh_real, agua_real_l]
        ax[1].bar(metricas, impacto, color=["#f44336","#3F51B5","#4CAF50"])
        ax[1].set_title("Consumo acumulado real")
        plt.tight_layout()
        plt.show()

        print(f"Gasto real actualizado para {nombre_usuario}:")
        for k,v in gasto_real.items():
            print(f"   {k.capitalize()}: {v:.0f} MXN")
        print(f"\n Ahorro restante total: {total_restante:.0f} MXN de {meta_state['total']:.0f}")

        if total_restante <= 0:
            print(" Ya has consumido todo tu ahorro planeado.")
        else:
            print("Todavía cumples con tu meta, pero vigila tus gastos.")


dropdown_user = Dropdown(options=df["Nombre"].tolist(), description="Usuario:")
meta_slider = FloatSlider(value=400, min=0, max=2000, step=10, description='Meta total')
gas_slider = FloatSlider(value=300, min=0, max=400, step=10, description='Gasolina')
agua_slider = FloatSlider(value=70, min=0, max=400, step=10, description='Agua')

monto_input = FloatText(value=0.0, description='Monto (MXN):')
btn_gas = Button(description="Registrar gasto gasolina", button_style='warning')
btn_luz = Button(description="Registrar gasto luz", button_style='info')
btn_agua = Button(description="Registrar gasto agua", button_style='success')

btn_gas.on_click(lambda x: registrar_gasto("gasolina", monto_input.value, dropdown_user.value))
btn_luz.on_click(lambda x: registrar_gasto("luz", monto_input.value, dropdown_user.value))
btn_agua.on_click(lambda x: registrar_gasto("agua", monto_input.value, dropdown_user.value))

#Interacción principal
interact(sim_usuario, nombre_usuario=dropdown_user, meta_total=meta_slider, ahorro_gas=gas_slider, ahorro_agua=agua_slider)
display(VBox([
    HBox([monto_input, btn_gas, btn_luz, btn_agua]),
    out
]))


Base de datos cargada correctamente.


interactive(children=(Dropdown(description='Usuario:', options=('Usuario1', 'Usuario2', 'Usuario3', 'Usuario4'…

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, FloatSlider, Button, VBox, HBox, Output, FloatText, Dropdown

PRECIO_LITRO_GASOLINA = 24.0
COSTO_KWH = 2.8
COSTO_M3_AGUA = 20.0
KGCO2_POR_LITRO = 2.31

df = pd.read_csv("/content/BaseDatos_SmartCity_Greedy_V3.csv")

#Validar que tenga columnas necesarias o crear aproximaciones
if "PromedioAcumuladoLuz" not in df.columns:
    df["PromedioAcumuladoLuz"] = np.random.uniform(200, 500, len(df))
if "PromedioAcumuladoAgua" not in df.columns:
    df["PromedioAcumuladoAgua"] = np.random.uniform(10, 35, len(df))
if "PromedioAcumuladoGasolina" not in df.columns:
    df["PromedioAcumuladoGasolina"] = np.random.uniform(300, 700, len(df))

print("Base cargada correctamente con", len(df), "registros")

def co2_from_gas_mxn(mxn, co2_km):
    litros = mxn / PRECIO_LITRO_GASOLINA
    return litros * KGCO2_POR_LITRO * (co2_km / 0.20)

def kwh_from_mxn(mxn): return mxn / COSTO_KWH
def m3_from_mxn(mxn):  return mxn / COSTO_M3_AGUA

out = Output()
meta_state = {"gas": 0, "agua": 0, "luz": 0, "total": 0}
gasto_real = {"gasolina": 0.0, "agua": 0.0, "luz": 0.0}

def sim_usuario(nombre_usuario, ahorro_gas, ahorro_agua):
    fila = df[df["Nombre"] == nombre_usuario].iloc[0]
    base_gas = fila["PromedioAcumuladoGasolina"]
    base_luz = fila["PromedioAcumuladoLuz"]
    base_agua = fila["PromedioAcumuladoAgua"]
    co2_km = fila["CO2porKm"]
    rend_kml = fila["RendimientoKmL"]

    #Luz = lo restante de los tres
    ahorro_gas = min(ahorro_gas, base_gas)
    ahorro_agua = min(ahorro_agua, base_agua)
    ahorro_luz = min(base_luz, base_luz * (1 - ((ahorro_gas/base_gas + ahorro_agua/base_agua)/2)))

    meta_total = ahorro_gas + ahorro_luz + ahorro_agua
    meta_state.update({"gas": ahorro_gas, "agua": ahorro_agua, "luz": ahorro_luz, "total": meta_total})

    #Porcentajes de reducción respecto a los promedios
    pct_gas = ahorro_gas / base_gas * 100
    pct_agua = ahorro_agua / base_agua * 100
    pct_luz = ahorro_luz / base_luz * 100

    #Impactos ambientales
    co2_ev = co2_from_gas_mxn(ahorro_gas, co2_km)
    kwh_ev = kwh_from_mxn(ahorro_luz)
    agua_ev_l = m3_from_mxn(ahorro_agua) * 1000

    #Visualización
    with out:
        out.clear_output()
        fig, ax = plt.subplots(1, 2, figsize=(10,4))

        rubros = ["Gasolina", "Luz", "Agua"]
        valores = [ahorro_gas, ahorro_luz, ahorro_agua]
        ax[0].bar(rubros, valores, color=["#FF9800","#2196F3","#00BCD4"])
        ax[0].set_title("Distribución de ahorro (MXN)")
        ax[0].set_ylim(0, max(base_gas, base_luz, base_agua)*1.1)

        metricas = ["CO₂ evitado (kg)", "kWh evitados", "Agua evitada (L)"]
        impacto = [co2_ev, kwh_ev, agua_ev_l]
        ax[1].bar(metricas, impacto, color=["#f44336","#3F51B5","#4CAF50"])
        ax[1].set_title("Impacto ecológico estimado")
        plt.tight_layout()
        plt.show()

        print(f" Usuario: {nombre_usuario}")
        print(f"Promedios globales → Gas: {base_gas:.0f}, Luz: {base_luz:.0f}, Agua: {base_agua:.0f}")
        print(f"\n Ahorro propuesto:")
        print(f"   Gasolina: {ahorro_gas:.0f} ({pct_gas:.1f}%)")
        print(f"   Agua: {ahorro_agua:.0f} ({pct_agua:.1f}%)")
        print(f"   Luz: {ahorro_luz:.0f} ({pct_luz:.1f}%)")
        print(f"\n Impacto ambiental reducido:")
        print(f"   CO₂ evitado: {co2_ev:.2f} kg")
        print(f"   Energía evitada: {kwh_ev:.1f} kWh")
        print(f"   Agua evitada: {agua_ev_l:.0f} L")


def registrar_gasto(cat, monto, nombre_usuario):
    fila = df[df["Nombre"] == nombre_usuario].iloc[0]
    co2_km = fila["CO2porKm"]

    gasto_real[cat] += monto
    ahorro_logrado = {
        "gas": max(0, meta_state["gas"] - gasto_real["gasolina"]),
        "agua": max(0, meta_state["agua"] - gasto_real["agua"]),
        "luz": max(0, meta_state["luz"] - gasto_real["luz"])
    }

    co2_real = co2_from_gas_mxn(gasto_real["gasolina"], co2_km)
    kwh_real = kwh_from_mxn(gasto_real["luz"])
    agua_real_l = m3_from_mxn(gasto_real["agua"]) * 1000

    total_restante = ahorro_logrado["gas"] + ahorro_logrado["agua"] + ahorro_logrado["luz"]

    with out:
        out.clear_output(wait=True)
        fig, ax = plt.subplots(1, 2, figsize=(10,4))
        rubros = ["Gasolina", "Luz", "Agua"]
        valores = [ahorro_logrado["gas"], ahorro_logrado["luz"], ahorro_logrado["agua"]]
        ax[0].bar(rubros, valores, color=["#FF9800","#2196F3","#00BCD4"])
        ax[0].set_title("Ahorro restante (MXN)")
        ax[0].set_ylim(0, meta_state["total"])

        metricas = ["CO₂ emitido (kg)", "kWh usados", "Agua usada (L)"]
        impacto = [co2_real, kwh_real, agua_real_l]
        ax[1].bar(metricas, impacto, color=["#f44336","#3F51B5","#4CAF50"])
        ax[1].set_title("Consumo acumulado real")
        plt.tight_layout()
        plt.show()

        print(f"Gasto real actualizado para {nombre_usuario}:")
        for k,v in gasto_real.items():
            print(f"   {k.capitalize()}: {v:.0f} MXN")
        print(f"\n Ahorro restante total: {total_restante:.0f} MXN de {meta_state['total']:.0f}")

        if total_restante <= 0:
            print("Ya has consumido todo tu ahorro planeado.")
        else:
            print("Todavía cumples con tu meta, pero vigila tus gastos.")

dropdown_user = Dropdown(options=df["Nombre"].tolist(), description="Usuario:")
meta_gas = FloatSlider(value=0, min=0, max=500, step=10, description='Ahorro Gasolina')
meta_agua = FloatSlider(value=0, min=0, max=200, step=5, description='Ahorro Agua')

monto_input = FloatText(value=0.0, description='Monto (MXN):')
btn_gas = Button(description="Registrar gasto gasolina", button_style='warning')
btn_luz = Button(description="Registrar gasto luz", button_style='info')
btn_agua = Button(description="Registrar gasto agua", button_style='success')

btn_gas.on_click(lambda x: registrar_gasto("gasolina", monto_input.value, dropdown_user.value))
btn_luz.on_click(lambda x: registrar_gasto("luz", monto_input.value, dropdown_user.value))
btn_agua.on_click(lambda x: registrar_gasto("agua", monto_input.value, dropdown_user.value))

interact(sim_usuario, nombre_usuario=dropdown_user, ahorro_gas=meta_gas, ahorro_agua=meta_agua)
display(VBox([
    HBox([monto_input, btn_gas, btn_luz, btn_agua]),
    out
]))


Base cargada correctamente con 40 registros


interactive(children=(Dropdown(description='Usuario:', options=('Usuario1', 'Usuario2', 'Usuario3', 'Usuario4'…

In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, FloatSlider, Button, VBox, HBox, Output, FloatText, Dropdown, interactive_output, Layout

PRECIO_LITRO_GASOLINA = 24.0
COSTO_KWH = 2.8
COSTO_M3_AGUA = 20.0
KGCO2_POR_LITRO = 2.31

df = pd.read_csv("/content/BaseDatos_SmartCity_Greedy_V3.csv")

cols_needed = ["Nombre","PromedioAcumuladoGasolina","PromedioAcumuladoLuz","PromedioAcumuladoAgua","CO2porKm","RendimientoKmL"]
for c in cols_needed:
    if c not in df.columns:
        raise ValueError(f"Falta columna '{c}' en la base de datos")

print("Base cargada correctamente con", len(df), "usuarios.")

def co2_from_gas_mxn(mxn, co2_km):
    litros = mxn / PRECIO_LITRO_GASOLINA
    return litros * KGCO2_POR_LITRO * (co2_km / 0.20)

def kwh_from_mxn(mxn): return mxn / COSTO_KWH
def m3_from_mxn(mxn):  return mxn / COSTO_M3_AGUA

out = Output()
meta_state = {"gas":0,"agua":0,"luz":0,"total":0}
gasto_real = {"gasolina":0.0,"agua":0.0,"luz":0.0}

def sim_usuario(nombre_usuario, ahorro_gas, ahorro_agua):
    fila = df[df["Nombre"] == nombre_usuario].iloc[0]
    base_gas = fila["PromedioAcumuladoGasolina"]
    base_luz = fila["PromedioAcumuladoLuz"]
    base_agua = fila["PromedioAcumuladoAgua"]
    co2_km = fila["CO2porKm"]

    #Luz se calcula automáticamente
    ahorro_luz = max(0, base_luz - ((ahorro_gas/base_gas + ahorro_agua/base_agua)/2)*base_luz)
    ahorro_luz = min(ahorro_luz, base_luz)
    meta_total = ahorro_gas + ahorro_luz + ahorro_agua
    meta_state.update({"gas":ahorro_gas,"agua":ahorro_agua,"luz":ahorro_luz,"total":meta_total})

    #Porcentajes
    pct_gas = ahorro_gas/base_gas*100
    pct_agua = ahorro_agua/base_agua*100
    pct_luz = ahorro_luz/base_luz*100

    #Impactos físicos
    co2_ev = co2_from_gas_mxn(ahorro_gas, co2_km)
    kwh_ev = kwh_from_mxn(ahorro_luz)
    agua_ev_l = m3_from_mxn(ahorro_agua)*1000

    with out:
        out.clear_output()
        fig, ax = plt.subplots(1, 2, figsize=(10,4))
        rubros = ["Gasolina","Luz","Agua"]
        valores = [ahorro_gas, ahorro_luz, ahorro_agua]
        ax[0].bar(rubros, valores, color=["#FF9800","#2196F3","#00BCD4"])
        ax[0].set_title("Distribución de ahorro planeado (MXN)")
        ax[0].set_ylim(0, max(base_gas,base_luz,base_agua)*1.2)

        metricas = ["CO₂ evitado (kg)","kWh evitados","Agua evitada (L)"]
        impacto = [co2_ev,kwh_ev,agua_ev_l]
        ax[1].bar(metricas, impacto, color=["#f44336","#3F51B5","#4CAF50"])
        ax[1].set_title("Impacto ecológico estimado")
        plt.tight_layout()
        plt.show()

        print(f"Usuario: {nombre_usuario}")
        print(f"Promedios → Gas: {base_gas:.1f}, Luz: {base_luz:.1f}, Agua: {base_agua:.1f}")
        print(f"\n Ahorro propuesto:")
        print(f"   Gasolina: {ahorro_gas:.0f} ({pct_gas:.1f}%)")
        print(f"   Agua: {ahorro_agua:.0f} ({pct_agua:.1f}%)")
        print(f"   Luz: {ahorro_luz:.0f} ({pct_luz:.1f}%)")
        print(f"\n Impacto reducido:")
        print(f"   CO₂ evitado: {co2_ev:.2f} kg")
        print(f"   Energía evitada: {kwh_ev:.1f} kWh")
        print(f"   Agua evitada: {agua_ev_l:.0f} L")


def actualizar_interfaz(nombre_usuario):
    """Regenera los sliders según el usuario seleccionado"""
    fila = df[df["Nombre"] == nombre_usuario].iloc[0]
    gas_max = float(fila["PromedioAcumuladoGasolina"])
    agua_max = float(fila["PromedioAcumuladoAgua"])
    luz_max = float(fila["PromedioAcumuladoLuz"])

    print(f"Actualizando sliders → Gasolina max: {gas_max:.2f}, Agua max: {agua_max:.2f}, Luz: {luz_max:.2f}")

    gas_slider = FloatSlider(value=0, min=0, max=gas_max, step=10, description='Gasolina', continuous_update=True)
    agua_slider = FloatSlider(value=0, min=0, max=agua_max, step=1, description='Agua', continuous_update=True)

    ui = VBox([gas_slider, agua_slider])
    out_func = interactive_output(sim_usuario, {'nombre_usuario':dropdown_user, 'ahorro_gas':gas_slider, 'ahorro_agua':agua_slider})
    display(ui, out_func)

dropdown_user = Dropdown(options=df["Nombre"].tolist(), description="Usuario:")

#Cuando cambie el usuario, se regeneran sliders dinámicamente
def on_user_change(change):
    if change["name"] == "value" and change["new"]:
        out.clear_output()
        actualizar_interfaz(change["new"])

dropdown_user.observe(on_user_change)

monto_input = FloatText(value=0.0, description='Monto (MXN):', layout=Layout(width='200px'))
btn_gas = Button(description="Registrar gasto gasolina", button_style='warning')
btn_luz = Button(description="Registrar gasto luz", button_style='info')
btn_agua = Button(description="Registrar gasto agua", button_style='success')

def registrar_gasto(cat):
    nombre = dropdown_user.value
    monto = monto_input.value
    fila = df[df["Nombre"] == nombre].iloc[0]
    co2_km = fila["CO2porKm"]

    gasto_real[cat] += monto
    ahorro_logrado = {
        "gas": max(0, meta_state["gas"] - gasto_real["gasolina"]),
        "agua": max(0, meta_state["agua"] - gasto_real["agua"]),
        "luz": max(0, meta_state["luz"] - gasto_real["luz"])
    }

    total_restante = ahorro_logrado["gas"] + ahorro_logrado["agua"] + ahorro_logrado["luz"]
    with out:
        out.clear_output(wait=True)
        print(f" {nombre} - Ahorro restante total: {total_restante:.2f} MXN")

btn_gas.on_click(lambda x: registrar_gasto("gasolina"))
btn_luz.on_click(lambda x: registrar_gasto("luz"))
btn_agua.on_click(lambda x: registrar_gasto("agua"))

display(VBox([
    dropdown_user,
    HBox([monto_input, btn_gas, btn_luz, btn_agua]),
    out
]))

#Iniciar interfaz con primer usuario
actualizar_interfaz(df["Nombre"].iloc[0])


Base cargada correctamente con 40 usuarios.


Actualizando sliders → Gasolina max: 507.20, Agua max: 20.49, Luz: 305.02


Output()

Actualizando sliders → Gasolina max: 2265.95, Agua max: 24.47, Luz: 465.05


Output()